In [5]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum, GurobiError

In [11]:
O = np.array([0,1,2,3,4])
I = np.array([0,1,2,3,4])
A = np.array([0,1,2,3,4])


u = {0: {0:3,1:0,2:1,3:0,4:0}, 1: {0:0,1:1,2:0,3:1,4:0}, 2:{0:0,1:0,2:1,3:0,4:2}, 3:{0:1,1:0,2:2,3:1,4:1}, 4:{0:0,1:1,2:0,3:0,4:0} }
p = {0:{0:2,1:1,2:1,3:0,4:1}, 1:{0:2,1:1,2:2,3:0,4:1}, 2: {0:0,1:2,2:0,3:1,4:2}, 3: {0:2,1:1,2:0,3:1,4:1}, 4:{0:0,1:1,2:2,3:1,4:2} }


modelo = Model("Modelo")

x = modelo.addVars(O, lb=0, ub=1, vtype=GRB.BINARY, name="x orden")
y = modelo.addVars(A, lb=0, ub=1, vtype=GRB.BINARY, name="y pasillo")

modelo.addConstrs(( sum(u[o][i] *x[o] for o in O ) <= sum( p[a][i]*y[a]  for a in A)    for i in  I   ))

modelo.addConstr(( sum(y[a] for a in A ) == 2 ))
modelo.setObjective(quicksum(x[o]*u[o][i] for o in O for i in I) , GRB.MAXIMIZE)

#modelo.setParam('OutputFlag', False)
modelo.optimize()



Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8500Y CPU @ 1.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 10 columns and 35 nonzeros
Model fingerprint: 0xbd1fea21
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 5 rows, 9 columns, 25 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)

Root relaxation: objective 1.100000e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.

In [10]:
modelo.ObjVal

15.0

In [8]:
for v in modelo.getVars():
    print(f"{v.VarName} = {v.X}")

x orden[0] = 1.0
x orden[1] = 1.0
x orden[2] = 1.0
x orden[3] = 0.0
x orden[4] = 1.0
y pasillo[0] = -0.0
y pasillo[1] = 1.0
y pasillo[2] = -0.0
y pasillo[3] = 1.0
y pasillo[4] = -0.0


In [1]:
from gurobipy import *

import numpy as np

O = np.array([0,1,2,3,4])
I = np.array([0,1,2,3,4])
A = np.array([0,1,2,3,4])

u = {
    0: {0:3,1:0,2:1,3:0,4:0},
    1: {0:0,1:1,2:0,3:1,4:0},
    2: {0:0,1:0,2:1,3:0,4:2},
    3: {0:1,1:0,2:2,3:1,4:1},
    4: {0:0,1:1,2:0,3:0,4:0}
}

p = {
    0: {0:2,1:1,2:1,3:0,4:1},
    1: {0:2,1:1,2:2,3:0,4:1},
    2: {0:0,1:2,2:0,3:1,4:2},
    3: {0:2,1:1,2:0,3:1,4:1},
    4: {0:0,1:1,2:2,3:1,4:2}
}

mejor_ratio = -1
mejor_solucion = {}

# Buscar para cada valor posible del denominador (de 1 a 5)
for D in range(1, 6):

    modelo = Model(f"Modelo_lineal_D={D}")
    modelo.setParam('OutputFlag', 0)  # No mostrar log

    x = modelo.addVars(O, vtype=GRB.BINARY, name="x")
    y = modelo.addVars(A, vtype=GRB.BINARY, name="y")

    # Restricciones de capacidad
    for i in I:
        modelo.addConstr(sum(u[o][i] * x[o] for o in O) <= sum(p[a][i] * y[a] for a in A))

    # Fijar la suma del denominador
    modelo.addConstr(sum(y[a] for a in A) == D)

    # Maximizar numerador
    modelo.setObjective(quicksum(x[o] * u[o][i] for o in O for i in I), GRB.MAXIMIZE)

    modelo.optimize()

    if modelo.status == GRB.OPTIMAL:
        numerador = modelo.objVal
        ratio = numerador / D

        if ratio > mejor_ratio:
            mejor_ratio = ratio
            mejor_solucion = {
                'D': D,
                'x': {o: x[o].X for o in O},
                'y': {a: y[a].X for a in A},
                'numerador': numerador,
                'objetivo': ratio
            }

# Mostrar la mejor solución encontrada
print(f"Mejor D: {mejor_solucion['D']}")
print(f"Mejor numerador: {mejor_solucion['numerador']}")
print(f"Mejor ratio objetivo (f.o.): {mejor_solucion['objetivo']:.4f}")
print("x =", mejor_solucion['x'])
print("y =", mejor_solucion['y'])


Set parameter Username
Set parameter LicenseID to value 2598815
Academic license - for non-commercial use only - expires 2025-12-12
Mejor D: 1
Mejor numerador: 5.0
Mejor ratio objetivo (f.o.): 5.0000
x = {np.int64(0): 0.0, np.int64(1): 1.0, np.int64(2): 1.0, np.int64(3): 0.0, np.int64(4): 0.0}
y = {np.int64(0): 0.0, np.int64(1): 0.0, np.int64(2): 0.0, np.int64(3): 0.0, np.int64(4): 1.0}
